In [2]:
!pip install pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("PySpark-jupyter-demo") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/27 11:36:56 WARN Utils: Your hostname, Noas-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.106 instead (on interface en0)
25/12/27 11:36:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/27 11:36:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Test the setup
data = [("Kia", "Niro", 2025), ("Toyota", "Rav4", 2019), ("BYD", "Atto3", 2024)]
df = spark.createDataFrame(data, ["Car", "Model", "Year"])
df.show()

+------+-----+----+
|   Car|Model|Year|
+------+-----+----+
|   Kia| Niro|2025|
|Toyota| Rav4|2019|
|   BYD|Atto3|2024|
+------+-----+----+



In [12]:
df_lines = spark.read.format("com.databricks.spark.csv").options(header='true') \
 .option("delimiter", ",").option("quote", '"') \
 .option("escape", '"').option("multiLine", True) \
 .load("/Users/noalevy/big_data/ex3/pyspark/simpsons/simpsons_script_lines.csv")
df_lines.createOrReplaceTempView("lines_table")
## print lines file schema
df_lines.printSchema()
## print all fields for first 5 rows
spark.sql("SELECT * from lines_table").show(5)
## print the shortest 10 script lines:
spark.sql("SELECT int(word_count) AS wc, raw_text FROM lines_table WHERE TRY_CAST(word_count AS INT) > 0 ORDER BY wc ").show(10,False)
## count the scenes that took more than 10 minutes
spark.sql("SELECT count(*) FROM lines_table WHERE TRY_CAST(timestamp_in_ms AS INT) > (10 * 60 * 1000)").show()

root
 |-- id: string (nullable = true)
 |-- episode_id: string (nullable = true)
 |-- number: string (nullable = true)
 |-- raw_text: string (nullable = true)
 |-- timestamp_in_ms: string (nullable = true)
 |-- speaking_line: string (nullable = true)
 |-- character_id: string (nullable = true)
 |-- location_id: string (nullable = true)
 |-- raw_character_text: string (nullable = true)
 |-- raw_location_text: string (nullable = true)
 |-- spoken_words: string (nullable = true)
 |-- normalized_text: string (nullable = true)
 |-- word_count: string (nullable = true)

+----+----------+------+--------------------+---------------+-------------+------------+-----------+--------------------+--------------------+--------------------+--------------------+----------+
|  id|episode_id|number|            raw_text|timestamp_in_ms|speaking_line|character_id|location_id|  raw_character_text|   raw_location_text|        spoken_words|     normalized_text|word_count|
+----+----------+------+-------------

## Ex1

#### Question 1

In [16]:
## print the first 50 locations that has the word "Springfield" in them , ignoring letters case.

df_locations = spark.read.format("com.databricks.spark.csv").options(header='true') \
 .option("delimiter", ",").option("quote", '"') \
 .option("escape", '"').option("multiLine", True) \
 .load("/Users/noalevy/big_data/ex3/pyspark/simpsons/simpsons_locations.csv")

df_locations.createOrReplaceTempView("locations_table")

df_result_first = spark.sql("SELECT name FROM locations_table WHERE normalized_name like '%springfield%'")

df_result_first = df_result_first.toPandas()

print(df_result_first.head(50))

                                               name
0                     Springfield Elementary School
1                                  Springfield Mall
2                   Springfield Nuclear Power Plant
3                       Springfield Downs Dog Track
4                                 SPRINGFIELD DOWNS
5                                  SPRINGFIELD DOWN
6                     SPRINGFIELD DOWNS PARKING LOT
7          Springfield Elementary School Playground
8                             Springfield Town Hall
9             Springfield Elementary School Hallway
10          Springfield Elementary School Cafeteria
11                    Springfield Retirement Castle
12   Grampa's Room at Springfield Retirement Castle
13                              Springfield Library
14               Springfield Elementary School Yard
15                      First Church of Springfield
16            Springfield Elementary School Library
17                              Springfield Airport
18          

#### Question 2

In [19]:
## print 20 quotes that are located in any place that has Jerusalem in its name.
## Note that Jerusalem may appear in any case and in any part of the location name.
## Use JOIN for this query on another table

df_simpsons_script_lines = spark.read.format("com.databricks.spark.csv").options(header='true') \
 .option("delimiter", ",").option("quote", '"') \
 .option("escape", '"').option("multiLine", True) \
 .load("/Users/noalevy/big_data/ex3/pyspark/simpsons/simpsons_script_lines.csv")

df_simpsons_script_lines.createOrReplaceTempView("quotes_table")

df_result_two = spark.sql("SELECT raw_text FROM quotes_table JOIN locations_table ON quotes_table.location_id = locations_table.id \
            WHERE normalized_name like '%jerusalem%'")

df_result_two = df_result_two.toPandas()

print(df_result_two.head(20))

                                             raw_text
0   (JERUSALEM ROYAL PALACE: ext. jerusalem royal ...
1         Bart Simpson: I'm bored. Send in my jester.
2   Krusty the Clown: Hey, hey, King David! (GOES ...
3                    Bart Simpson: What else you got?
4   Krusty the Clown: Ahh... yeah... well, wait a ...
5                  Methuselah Grampa: (PAINED SOUNDS)
6   Bart Simpson: Methuselah, my oldest friend. Wh...
7   Methuselah Grampa: (WEAK, PAINED GROAN) It was...
8   Bart Simpson: (PUZZLED) But Goliath is dead. I...
9   Methuselah Grampa: No, it was his son, Goliath...
10  Bart Simpson: (MELODRAMATIC) Nooo... (THEN, ST...
11      (Jerusalem: EXT. JERUSALEM - FIELD OF BATTLE)
12  Bart Simpson: (COCKY) I'll just give Goliath T...
13  Santa's Little Helper: (DOPEY "DAVEY AND GOLIA...
14                Bart Simpson: (ANNOYED) Quiet, you.
15  Santa's Little Helper: Oh, you've gotten prett...
16       Goliath/nelson: (OMINOUS) Let's get it on...
17        Santa's Little Hel

#### Question 3

In [22]:
## print first 20 most used locations with the count of lines spoken in them.
## use GROUP BY for that

df_result_third = spark.sql("SELECT locations_table.normalized_name AS location, \
            COUNT(*) AS line_count FROM quotes_table \
            JOIN locations_table ON quotes_table.location_id = locations_table.id \
            GROUP BY locations_table.normalized_name \
            ORDER BY line_count DESC \
            LIMIT 20 \
            ")

df_result_third = df_result_third.toPandas()

print(df_result_third.head(20))

                           location  line_count
0                      simpson home       35059
1     springfield elementary school        7092
2                        moe tavern        4628
3   springfield nuclear power plant        3594
4                       kwik-e-mart        1476
5       first church of springfield        1416
6               simpson living room        1378
7                springfield street        1334
8                       springfield        1314
9                       simpson car        1239
10                    flanders home        1166
11                           street        1124
12            springfield town hall        1103
13    springfield retirement castle        1049
14                      burns manor         998
15                 springfield mall         833
16                  simpson kitchen         816
17                        courtroom         813
18                   bart treehouse         767
19                     bart bedroom     

#### Question 4

In [25]:
## find the seasons in which the average imdb rating was the highest.
## Print the seasons number, the number of episodes in each one and the average rating
## in a descending order from highest average rating to lowest.

df_simpsons_episodes = spark.read.format("com.databricks.spark.csv").options(header='true') \
 .option("delimiter", ",").option("quote", '"') \
 .option("escape", '"').option("multiLine", True) \
 .load("/Users/noalevy/big_data/ex3/pyspark/simpsons/simpsons_episodes.csv")

df_simpsons_episodes.createOrReplaceTempView("episodes_table")

df_result_fourth = spark.sql("SELECT season, COUNT(*) AS number_of_episodes, AVG(imdb_rating) AS imdb_avg_rating \
            FROM episodes_table \
            GROUP BY season \
            ORDER BY AVG(imdb_rating) DESC ")

df_result_fourth = df_result_fourth.toPandas()

print(df_result_fourth)

   season  number_of_episodes  imdb_avg_rating
0       5                  22         8.336364
1       7                  25         8.324000
2       6                  25         8.312000
3       4                  22         8.268182
4       8                  25         8.220000
5       3                  24         8.154167
6       2                  22         8.040909
7       9                  25         7.844000
8       1                  13         7.807692
9      10                  23         7.569565
10     12                  21         7.361905
11     11                  22         7.290909
12     13                  22         7.140909
13     14                  22         7.077273
14     15                  22         7.045455
15     16                  21         7.042857
16     18                  22         7.000000
17     19                  20         6.935000
18     20                  21         6.895238
19     17                  22         6.863636
20     25    

## EX 2

In [28]:
## The query is - which of the characters appears the most in the episodes titles?
## The characters that are being checked are - Homer, Lisa, Bart, Maggie, Marge

df_ex2 = spark.sql("SELECT 'Homer' AS character_name, COUNT(*) AS title_appearances \
                    FROM episodes_table \
                    WHERE title LIKE '%Homer%' \
                    UNION ALL \
                    SELECT 'Bart' AS character_name, COUNT(*) AS title_appearances \
                    FROM episodes_table \
                    WHERE title LIKE '%Bart%' \
                    UNION ALL \
                    SELECT 'Maggie' AS character_name, COUNT(*) AS title_appearances \
                    FROM episodes_table \
                    WHERE title LIKE '%Maggie%' \
                    UNION ALL \
                    SELECT 'Lisa' AS character_name, COUNT(*) AS title_appearances \
                    FROM episodes_table \
                    WHERE title LIKE '%Lisa%' \
                    UNION ALL \
                    SELECT 'Marge' AS character_name, COUNT(*) AS title_appearances \
                    FROM episodes_table \
                    WHERE title LIKE '%Marge%' \
                    ORDER BY title_appearances DESC \
                    ")

df_ex2 = df_ex2.toPandas()

print(df_ex2)

  character_name  title_appearances
0          Homer                 55
1           Bart                 42
2           Lisa                 38
3          Marge                 20
4         Maggie                  2
